# Counterfactual simulation with adaptive responses and QALY statistics
This notebook demonstrates how to do a counterfactual simulation across different adaptive interventions and detection times in pathosim. It also includes ways to get QALY statistics from the results.

In [1]:
import unittest 
import pathosim as inf
import sciris as sc
import os
import numpy as np
    
result_keys = [ 
    'n_infectious',  
    'n_symptomatic', 
    'n_severe',       
    'n_recovered',   
    'n_dead']

PathoSim 3.1.2 (2022-01-16) — © 2023 by McGill University


##### Inputs

Life expectancy table for the UK population (taken from National life tables 2020 to 2022, Office for National Statistics). We use the expected residual life (remaining years) at a given age (from age 0 to 100). This allows us to measure the burden of disease in metrics such as Years of Life Lost (YLL).

In [2]:
le_UK = inf.LifeExpectancy(
    years_remaining_males=[
        78.57, 77.91, 76.93, 75.94, 74.95, 73.96, 72.96, 71.97, 70.97, 69.98,
        68.98, 67.99, 66.99, 66.00, 65.01, 64.01, 63.02, 62.04, 61.06, 60.08,
        59.11, 58.14, 57.17, 56.20, 55.23, 54.26, 53.29, 52.33, 51.36, 50.39,
        49.43, 48.47, 47.51, 46.55, 45.60, 44.65, 43.69, 42.75, 41.81, 40.86,
        39.93, 39.00, 38.07, 37.14, 36.22, 35.30, 34.39, 33.48, 32.58, 31.69,
        30.80, 29.92, 29.04, 28.17, 27.30, 26.43, 25.58, 24.73, 23.89, 23.05,
        22.23, 21.41, 20.61, 19.81, 19.03, 18.25, 17.48, 16.73, 15.99, 15.26,
        14.54, 13.84, 13.14, 12.46, 11.78, 11.12, 10.47, 9.86, 9.26, 8.68,
        8.12, 7.60, 7.08, 6.60, 6.12, 5.68, 5.25, 4.85, 4.48, 4.14, 3.81,
        3.52, 3.25, 3.00, 2.78, 2.57, 2.40, 2.22, 2.07, 1.91, 1.8
],
    years_remaining_females=[
        82.57, 81.86, 80.88, 79.89, 78.90, 77.90, 76.91, 75.91, 74.92, 73.92,
        72.92, 71.93, 70.93, 69.94, 68.94, 67.95, 66.96, 65.97, 64.98, 63.99,
        63.00, 62.01, 61.03, 60.04, 59.06, 58.07, 57.08, 56.10, 55.12, 54.13,
        53.15, 52.17, 51.19, 50.21, 49.24, 48.27, 47.30, 46.33, 45.36, 44.40,
        43.44, 42.48, 41.53, 40.58, 39.63, 38.68, 37.74, 36.80, 35.87, 34.94,
        34.02, 33.10, 32.18, 31.27, 30.36, 29.46, 28.56, 27.66, 26.78, 25.89,
        25.02, 24.15, 23.29, 22.44, 21.60, 20.76, 19.93, 19.12, 18.31, 17.51,
        16.72, 15.94, 15.17, 14.41, 13.66, 12.92, 12.20, 11.51, 10.83, 10.16,
        9.53, 8.91, 8.32, 7.75, 7.20, 6.67, 6.17, 5.70, 5.26, 4.85, 4.47, 4.11,
        3.78, 3.49, 3.23, 2.97, 2.74, 2.54, 2.35, 2.20, 2.04
])


##### Baseline simulation parameters

In [3]:
sim_pars = dict(
        use_waning    = True,           
        pop_size      = 5000,       
        pop_type      = 'behaviour_module',      
        n_days        = 100,            
        verbose       = 0,             
        rand_seed     = 42,
        burden        = inf.Burden(life_expectancy=le_UK) # this will compute burden/utility statistics and report them in results and summary                
    )  

##### Intervention packages
We define two different intervention packages which each consist of several interventions. The effect of each intervention is specified in:

"data/effects_npis.json"
"data/effects_therapeutics.json"

In [4]:
# intervention packages (stored as dict)
packages = {
    "medium" : [inf.intervention_bucket(strength='medium')],
    "strong" : [inf.intervention_bucket(strength='´strong')]
}

##### Events

The events are chosen to decide which interventions are to be considered, and how/when. There are currently 4 main ways to do this, represented by 4 different Event classes:

* FixedEvent - the event is scheduled for a predetermined time (parameters: name, t_event)
* DelayedTrigger - the event is scheduled at a certain time after another event, the "trigger" (parameters: name, start_delay, stop_delay, trigger_event)
* RatioEvent - schedules a start (and stop) event at a certain ratio of infections of the susceptible population (parameters: name, start_delay, stop_delay, start_threshold, stop_threshold)
* RateEvent - schedules a start (and stop) event at a certain rate of increase in cases (parameters: name, start_delay, stop_delay, start_threshold, stop_threshold)

In [5]:
# First, create the detection event
detection_event = inf.FixedEvent('detection', 15) # let the detection event be fixed
#detection_event = inf.RatioEvent('detection', start_delay=0, start_threshold=0.001) # let the detection event be contingent

# Then, create events for the interventions to be considered
school_closure_ev = inf.RateEvent('school closure', start_delay=7, stop_delay=7, start_threshold=0.001, stop_threshold=0.0002)
home_office_ev = inf.RateEvent('home office', start_delay=2, stop_delay=2, start_threshold=0.001, stop_threshold=0.0002)
mask_mandate_ev = inf.DelayedTrigger('mask mandate', start_delay=5, stop_delay=None, trigger_event='detection')

all_events = [detection_event, school_closure_ev, home_office_ev, mask_mandate_ev]

##### Pathogen

In [6]:
pathogen = inf.SARS_COV_2(10)
pathogen.configure_hrql_loss(loss_symptomatic=0.43/365, loss_severe=(0.43+0.5)/365, loss_critical=(0.43+0.6)/365)

##### Set up counterfactual simulation

In [7]:
cf = inf.CounterfactualMultiSim(sim_pars, pathogens = [pathogen], events=all_events, intervention_packages=packages, n_sims=5, maxcpu=0.9, maxmem = 0.9, parallelize = True)

Run the baseline scenario (no interventions). This also checks whether a larger epidemic occurs in the simulated scenario and determines a realistic range of detection times.

In [8]:
cf.run_baseline(verbose = True)

In [15]:
i = 2
print(cf.sims[i].sim_baseline)
print(f"Detection range: {cf.sims[i].sim_baseline.get_detection_ranges()}")
print(f"Is epidemic: {cf.sims[i].sim_baseline.is_epidemic()}")

Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 9322⚙, 36☠)
Detection range: [{'lower': 4.0, 'upper': 27}]
Is epidemic: [True]


You can also get a summary of the baseline simulation in the form of a pandas dataframe.

In [16]:
cf.sims[i].summary_baseline

,pathogen,cum_infections,cum_reinfections,cum_infectious,cum_symptomatic,cum_severe,cum_critical,cum_recoveries,cum_deaths,cum_tests,...,rel_test_yield,frac_vaccinated,pop_imm,pop_nabs,pop_protection,pop_symp_protection,new_diagnoses_custom,cum_diagnoses_custom,tot_yll,tot_hrql_loss
0,0,9322.0,4330.0,9233.0,5262.0,351.0,109.0,8975.0,36.0,0.0,...,0.0,0.0,0.0,4.514877,0.825653,0.32994,0.0,0.0,457.47,66.179096


##### Run counterfactual simulations
To run a counterfactual simulation, you have to specify which intervention package should be used and what detection time to assume.

The simulations are parallelized across seeds. (Parallelized simulation is probably not compatible with current implementation of events)

In [13]:
#cf.run_counterfactual(intervention_package_key="medium", detection_times=[2, 4], verbose = True)

Note that for debugging / testing purposes, you can also run simulations in non-parallel mode:

In [20]:
cf.run_counterfactual(intervention_package_key="medium", detection_times=[2, 4], verbose = True, parallelize=False)

Running counterfactual (seed 42) for intervention package "medium" with detection time 2.
Running counterfactual (seed 42) for intervention package "medium" with detection time 4.
Running counterfactual (seed 43) for intervention package "medium" with detection time 2.
Running counterfactual (seed 43) for intervention package "medium" with detection time 4.
Running counterfactual (seed 44) for intervention package "medium" with detection time 2.
Running counterfactual (seed 44) for intervention package "medium" with detection time 4.
Running counterfactual (seed 45) for intervention package "medium" with detection time 2.
Running counterfactual (seed 45) for intervention package "medium" with detection time 4.
Running counterfactual (seed 46) for intervention package "medium" with detection time 2.
Running counterfactual (seed 46) for intervention package "medium" with detection time 4.


You can inspect the stored counterfactual simulations in `cf.sims_counterfactual`.

In [19]:
i = 4
cf.sims[i].sims_counterfactual

#0: 'medium': {2: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 9196⚙, 38☠), 4: Sim(<no label>; 2020-03-01 to 2020-06-09;
pop: 5000 behaviour_module; epi: 9196⚙, 38☠)}
#1: 'strong': {}


This is a two-level dictionary. The first key represents the intervention package, and the second key the detection time.

In [41]:
i = 0
cf.sims[i].sims_counterfactual["medium"][2]

Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠)

The summary of each counterfactual run is stored in an attribute called `summaries_counterfactual`. It is again a two-level dictionary with the same logic as the `sims_counterfactual` dictionary.

In [42]:
cf.sims[i].summaries_counterfactual["medium"][4]

,pathogen,cum_infections,cum_reinfections,cum_infectious,cum_symptomatic,cum_severe,cum_critical,cum_recoveries,cum_deaths,cum_tests,...,rel_test_yield,frac_vaccinated,pop_imm,pop_nabs,pop_protection,pop_symp_protection,new_diagnoses_custom,cum_diagnoses_custom,tot_yll,tot_hrql_loss
0,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,8407.0,28.0,0.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671


But it is much more convenient to get a pandas data frame with all results and across all seeds:

In [43]:
cf.get_summaries_df()

,seed,intervention_package,delay,pathogen,cum_infections,cum_reinfections,cum_infectious,cum_symptomatic,cum_severe,cum_critical,...,rel_test_yield,frac_vaccinated,pop_imm,pop_nabs,pop_protection,pop_symp_protection,new_diagnoses_custom,cum_diagnoses_custom,tot_yll,tot_hrql_loss
0,42,baseline,0,0,9284.0,4289.0,9169.0,5309.0,328.0,107.0,...,0.0,0.0,0.0,4.397622,0.819455,0.329671,0.0,0.0,482.47,65.579699
0,42,medium,2,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671
0,42,medium,4,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671
0,43,baseline,0,0,9424.0,4428.0,9282.0,5327.0,338.0,114.0,...,0.0,0.0,0.0,4.634543,0.826632,0.330230,0.0,0.0,508.46,66.920658
0,43,medium,2,0,8473.0,3484.0,8419.0,4917.0,307.0,107.0,...,0.0,0.0,0.0,4.233403,0.804375,0.328525,0.0,0.0,488.68,60.797014
0,43,medium,4,0,8473.0,3484.0,8419.0,4917.0,307.0,107.0,...,0.0,0.0,0.0,4.233403,0.804375,0.328525,0.0,0.0,488.68,60.797014
0,44,baseline,0,0,9322.0,4330.0,9233.0,5262.0,351.0,109.0,...,0.0,0.0,0.0,4.514877,0.825653,0.329940,0.0,0.0,457.47,66.179096
0,44,medium,2,0,8446.0,3459.0,8376.0,4875.0,329.0,104.0,...,0.0,0.0,0.0,4.182454,0.808790,0.328658,0.0,0.0,334.61,61.025479
0,44,medium,4,0,8446.0,3459.0,8376.0,4875.0,329.0,104.0,...,0.0,0.0,0.0,4.182454,0.808790,0.328658,0.0,0.0,334.61,61.025479
0,45,baseline,0,0,9024.0,4027.0,8955.0,5236.0,331.0,93.0,...,0.0,0.0,0.0,4.395917,0.813721,0.329556,0.0,0.0,362.46,65.082712


If you only want results for a specific seed, you can of course access the underlying object:

In [45]:
cf.sims[1].get_summaries_df()

,seed,intervention_package,delay,pathogen,cum_infections,cum_reinfections,cum_infectious,cum_symptomatic,cum_severe,cum_critical,...,rel_test_yield,frac_vaccinated,pop_imm,pop_nabs,pop_protection,pop_symp_protection,new_diagnoses_custom,cum_diagnoses_custom,tot_yll,tot_hrql_loss
0,43,baseline,0,0,9424.0,4428.0,9282.0,5327.0,338.0,114.0,...,0.0,0.0,0.0,4.634543,0.826632,0.330230,0.0,0.0,508.46,66.920658
0,43,medium,2,0,8473.0,3484.0,8419.0,4917.0,307.0,107.0,...,0.0,0.0,0.0,4.233403,0.804375,0.328525,0.0,0.0,488.68,60.797014
0,43,medium,4,0,8473.0,3484.0,8419.0,4917.0,307.0,107.0,...,0.0,0.0,0.0,4.233403,0.804375,0.328525,0.0,0.0,488.68,60.797014


##### Scan detection times with counterfactual simulation

This performs a scan over the detection range identified from the baseline simulation. The range is divided into parts according to `n_steps`.

In [46]:
cf.scan_detection_range(intervention_package_keys="medium", n_steps = 3, verbose = True)

In [47]:
i = 0
cf.sims[i].sims_counterfactual

#0: 'medium': {2: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 8702⚙, 28☠), 4: Sim(<no label>; 2020-03-01 to 2020-06-09;
pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 8: Sim(<no label>; 2020-03-01 to
2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 16: Sim(<no label>;
2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 23:
Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi:
8702⚙, 28☠)}
#1: 'strong': {}


In [48]:
cf.get_summaries_df()

,seed,intervention_package,delay,pathogen,cum_infections,cum_reinfections,cum_infectious,cum_symptomatic,cum_severe,cum_critical,...,rel_test_yield,frac_vaccinated,pop_imm,pop_nabs,pop_protection,pop_symp_protection,new_diagnoses_custom,cum_diagnoses_custom,tot_yll,tot_hrql_loss
0,42,baseline,0,0,9284.0,4289.0,9169.0,5309.0,328.0,107.0,...,0.0,0.0,0.0,4.397622,0.819455,0.329671,0.0,0.0,482.47,65.579699
0,42,medium,2,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671
0,42,medium,4,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671
0,42,medium,8,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671
0,42,medium,16,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671
0,42,medium,23,0,8702.0,3715.0,8611.0,5033.0,321.0,97.0,...,0.0,0.0,0.0,4.256794,0.805765,0.328463,0.0,0.0,306.39,62.373671
0,43,baseline,0,0,9424.0,4428.0,9282.0,5327.0,338.0,114.0,...,0.0,0.0,0.0,4.634543,0.826632,0.330230,0.0,0.0,508.46,66.920658
0,43,medium,2,0,8473.0,3484.0,8419.0,4917.0,307.0,107.0,...,0.0,0.0,0.0,4.233403,0.804375,0.328525,0.0,0.0,488.68,60.797014
0,43,medium,4,0,8473.0,3484.0,8419.0,4917.0,307.0,107.0,...,0.0,0.0,0.0,4.233403,0.804375,0.328525,0.0,0.0,488.68,60.797014
0,43,medium,5,0,8473.0,3484.0,8419.0,4917.0,307.0,107.0,...,0.0,0.0,0.0,4.233403,0.804375,0.328525,0.0,0.0,488.68,60.797014


You can also leave out `intervention_package_keys` (or set it to `None`), in which case the detection time scan will be run for all packages.

In [49]:
cf.scan_detection_range(n_steps = 3, verbose = True)

In [50]:
i = 0
cf.sims[i].sims_counterfactual

#0: 'medium': {2: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 8702⚙, 28☠), 4: Sim(<no label>; 2020-03-01 to 2020-06-09;
pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 8: Sim(<no label>; 2020-03-01 to
2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 16: Sim(<no label>;
2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 8702⚙, 28☠), 23:
Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi:
8702⚙, 28☠)}
#1: 'strong': {8: Sim(<no label>; 2020-03-01 to 2020-06-09; pop: 5000
behaviour_module; epi: 7908⚙, 32☠), 16: Sim(<no label>; 2020-03-01 to
2020-06-09; pop: 5000 behaviour_module; epi: 7908⚙, 32☠), 23: Sim(<no label>;
2020-03-01 to 2020-06-09; pop: 5000 behaviour_module; epi: 7908⚙, 32☠)}


In [51]:
cf.get_summaries_df()[["seed", "intervention_package", "cum_deaths"]]

,seed,intervention_package,cum_deaths
0,42,baseline,41.0
0,42,medium,28.0
0,42,medium,28.0
0,42,medium,28.0
0,42,medium,28.0
0,42,medium,28.0
0,42,strong,32.0
0,42,strong,32.0
0,42,strong,32.0
0,43,baseline,38.0


If you run a lot of simulations, it is advisable to use the argument `store_sims = False`. In this case, only the summaries are stored to reduce the overall memory demand.

In [52]:
cf.scan_detection_range(n_steps = 10, store_sims = False, verbose = True)

In [ ]:
cf.get_summaries_df()

In [42]:

110 - cf.sims[0].sim_baseline.people.age[cf.sims[0].sim_baseline.people.dead]

array([37.52602823, 35.55678665, 29.20185033, 29.89881884, 27.04850464,
       48.77354953, 36.47482739, 12.8387236 , 28.25648595, 21.21245089,
       30.25671921, 41.14257563, 35.49387989, 38.74634087, 20.71021272,
       40.64594788, 38.89203285, 45.87195664, 22.17413372, 27.57252054,
       35.58801733, 27.74331984, 44.2186933 , 28.913713  , 22.31038484,
       15.11758871, 22.0188446 , 49.03711951, 42.09619454, 36.7058265 ,
       19.35625116, 26.86573189, 28.59199695, 24.68340377, 33.51922766,
       35.93368849, 19.55090503, 49.79930306, 20.86820395, 38.55031453,
       15.72298829])